In [ ]:
import re
import seaborn as sns
import pandas as pd
import numpy as np
import typing
import glob
import os
import copy
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
re_events=re.compile(r'^S(\d),1,(\d+),,uncore_cha_(\d+)/event=(.+),umask=(.+)/,\d+,\d+\.\d*,,$')

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('core-layout'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
data = pd.DataFrame()

for file in glob.glob(f'{experiment.path}/*.out'):
    cpu = int(os.path.splitext(os.path.basename(file))[0])

    for line in open(file).readlines():
        is_event_info = re_events.match(line)
        if not is_event_info:
            continue

        socket = int(is_event_info.group(1))
        value = int(is_event_info.group(2))
        cha_box_id = int(is_event_info.group(3))
        event = is_event_info.group(4)
        umask = is_event_info.group(5)

        data = pd.concat([
            data,
            pd.DataFrame([
                {
                    'cpu': cpu,
                    'socket': socket,
                    'value': value,
                    'cha_box_id': cha_box_id,
                    'event': event,
                    'umask': umask
                }
            ])
        ], ignore_index=True)

data

In [ ]:
# find cbos for each cpu. we check that with 144, which should increase at the cbo that belongs to the active core
# check vertical mesh egress

# TxR_VERT_OCCUPANCY0
# • Title:
# • Category: Vertical Egress Events
# • Event Code: 0x90
# • Register Restrictions :
# • Definition: Occupancy event for the egress buffers in the common mesh stop. The
#   egress is used to queue up requests destined for the vertical ring on the mesh.

TxR_VERT_OCCUPANCY0 = data[data['event'] == '144']
TxR_VERT_OCCUPANCY0

In [ ]:
sns.lineplot(data=TxR_VERT_OCCUPANCY0, x='cha_box_id', y='value', hue='cpu', legend='full', size='socket', palette=sns.color_palette())

In [ ]:

# TxR_HORZ_OCCUPANCY
# • Title:
# • Category: Horizontal Egress Events
# • Event Code: 0xA0
# • Register Restrictions :
# • Definition: Occupancy event for the transgress buffers in the common mesh stop.
#   The egress is used to queue up requests destined for the horizontal ring on the mesh.

TxR_HORZ_OCCUPANCY = data[data['event'] == '160']
TxR_HORZ_OCCUPANCY

In [ ]:
sns.lineplot(data=TxR_HORZ_OCCUPANCY, x='cha_box_id', y='value', hue='cpu', legend='full', size='socket', palette=sns.color_palette())

In [ ]:
# find CHA for each cpu. we check that with TxR_VERT_OCCUPANCY0/0x3, which should increase at the cbo that belongs to the active core
# \arg socket: the socket for which to get the mapping
def get_cha_to_cpu_map(socket: int) -> dict:
    TxR_VERT_OCCUPANCY0 = data[data['event'] == '144']
    TxR_VERT_OCCUPANCY0 = TxR_VERT_OCCUPANCY0[TxR_VERT_OCCUPANCY0['umask'] == '0x03']
    TxR_VERT_OCCUPANCY0 = TxR_VERT_OCCUPANCY0[TxR_VERT_OCCUPANCY0['socket'] == socket]

    cha_to_cpu_map = dict()

    for cha_box in TxR_VERT_OCCUPANCY0['cha_box_id'].unique():
        values = TxR_VERT_OCCUPANCY0[TxR_VERT_OCCUPANCY0['cha_box_id'] == cha_box]
        max_index = values['value'].idxmax()
        loc = TxR_VERT_OCCUPANCY0.loc[max_index]
        if loc['value'] > 2.0E8:
            cha_to_cpu_map[cha_box] = loc['cpu']

    return cha_to_cpu_map

In [ ]:
get_cha_to_cpu_map(0)

In [ ]:
# TxR_HORZ_OCCUPANCY
# • Title:
# • Category: Horizontal Egress Events
# • Event Code: 0xA0
# • Register Restrictions :
# • Definition: Occupancy event for the transgress buffers in the common mesh stop.
#   The egress is used to queue up requests destined for the horizontal ring on the mesh.

# AD (Address) - Uncredited and AK (Aknowledge)

# Find the CHA boxes of each row for a specified socket. The resulting rows will not be ordered.
# \arg socket: the socket for which to get the rows of CHA boxes
def get_cha_rows(socket: int) -> typing.List[typing.Set[np.int64]]:
    TxR_HORZ_OCCUPANCY = data[data['event'] == '160']
    TxR_HORZ_OCCUPANCY = TxR_HORZ_OCCUPANCY[TxR_HORZ_OCCUPANCY['umask'] == '0x03']
    TxR_HORZ_OCCUPANCY = TxR_HORZ_OCCUPANCY[TxR_HORZ_OCCUPANCY['socket'] == socket]
    TxR_HORZ_OCCUPANCY = TxR_HORZ_OCCUPANCY[TxR_HORZ_OCCUPANCY['value'] > 2.0E8]

    rows = list()

    for cpu in TxR_HORZ_OCCUPANCY['cpu'].unique():
        row_values = TxR_HORZ_OCCUPANCY[TxR_HORZ_OCCUPANCY['cpu'] == cpu]
        row = set(row_values['cha_box_id'])
        if row not in rows:
            rows.append(row)

    return rows

In [ ]:
get_cha_rows(0)

In [ ]:
# We assume that CHA boxes are numbered in a manner that each tile can be described by a range of CHA boxes.
# This means that the minimum cha box index of the second tile is bigger than the maximum cha box index of the first tile
# Get the rows that belong to a tile by looping over the rows of the tile and discovering new rows by incresing the index by which cha boxes are found.
# \arg rows: all rows of a socket
# \arg min_find_index: the index where to start searching for rows that belong to one tile.
def get_rows_of_tile(rows: typing.List[typing.Set[np.int64]], min_find_index: int) -> typing.List[typing.Set[np.int64]]:
    current_find_index = min_find_index
    filtered_rows = list()

    # Repeat four times since we a looking to find four rows
    for _i in range(4):
        for row in rows:
            if min_find_index <= min(row) <= current_find_index:
                if row not in filtered_rows:
                    filtered_rows.append(row)
                    current_find_index = max(row)

    return filtered_rows


In [ ]:
# Get the rows of the first tile of socket 0
get_rows_of_tile(get_cha_rows(0), 0)

In [ ]:
# Split the rows of chas into a list of tiles that each contains a list of rows with chas
# \arg cha_rows: The list of CHA boxes split into rows
def get_tiled_cha_rows(cha_rows: typing.List[typing.Set[np.int64]]) -> typing.List[typing.List[typing.Set[np.int64]]]:
    tiled_cha_rows = list()
    index = 0
    while index == 0 or len(rows) > 0:
        rows = get_rows_of_tile(get_cha_rows(0), index)
        if len(rows) == 0:
            break
        index = max(map(max, rows)) + 1
        tiled_cha_rows.append(rows)

    return tiled_cha_rows

In [ ]:
# Print the list of all CHA boxes in rows for each tiles of socket 0
for tile in get_tiled_cha_rows(get_cha_rows(0)):
    print(tile)

In [ ]:
def print_tile(cha_matrix: typing.List[typing.List[np.int64]], tile_index: int, cha_to_cpu_map: dict) -> typing.NoReturn:
    updated_cha_matrix = copy.deepcopy(cha_matrix)

    for row_index, row in enumerate(updated_cha_matrix):
        for column_index, column in enumerate(row):
            if column == -1:
                # Fill missing spaces with X
                updated_cha_matrix[row_index][column_index] = "X"
            else:
                # Fill CHAs with CPUs
                if column in cha_to_cpu_map:
                    updated_cha_matrix[row_index][column_index] = f"CHA: {column} CPU: {cha_to_cpu_map[column]}"
                else:
                    updated_cha_matrix[row_index][column_index] = f"CHA: {column}"

    # Fill missing memory controllers
    if tile_index == 0:
        updated_cha_matrix[0][1] = "M"
    elif tile_index == 1:
        updated_cha_matrix[0][2] = "M"
    elif tile_index == 2:
        updated_cha_matrix[3][1] = "M"
    elif tile_index == 3:
        updated_cha_matrix[3][2] = "M"
    else:
        assert("Tile indices bigger that 3 are not valid")

    for row in updated_cha_matrix:
        print(row)

In [ ]:
socket_to_tiles = {}

for socket in data['socket'].unique():
    cha_rows = get_cha_rows(socket)
    tiled_cha_rows = get_tiled_cha_rows(cha_rows)

    all_tiles = list()

    for tile_cha_rows in tiled_cha_rows:
        tile_cha_rows = sorted(list(map(lambda row: sorted(list(row)), tile_cha_rows)))

        # Each tile is a 4x4 matrix with the physical locations
        tile = list()
        for i in range(4):
            tile.append(4 * [ -1 ])

        # the index of the last row with four entries
        last_full_row_index = -1
        # emplace rows with all four entries into the tile matrix
        for index, row in enumerate(tile_cha_rows):
            if len(row) != 4:
                continue

            tile[index] = row
            last_full_row_index = index

        if last_full_row_index == -1:
            assert("We did not find any rows with four elements in the current tile!")

        print(f"Socket {socket}, Tile filed only with rows that contain all (four) CHA boxes")
        print(tile)

        # emplace rows that do not contain all four entries.
        # this code is quite hacky, especially the detection of the placing of the rows relative to the last full row.
        # it should ideally be replaced by an ILP formulation.
        for index, row in enumerate(tile_cha_rows):
            if len(row) == 4:
                continue

            if index < last_full_row_index:
                for current_row_column_index in range(len(row)):
                    for absolute_column_index in range(4):
                        if row[current_row_column_index] < tile[last_full_row_index][absolute_column_index]:
                            tile[index][absolute_column_index] = row[current_row_column_index]
                            break

            if index > last_full_row_index:
                for current_row_column_index in reversed(range(len(row))):
                    for absolute_column_index in range(4):
                        if row[current_row_column_index] > tile_cha_rows[last_full_row_index][absolute_column_index]:
                            tile[index][absolute_column_index] = row[current_row_column_index]

                            # If the entry was emplaced into the previous column, keep the current and delete the last
                            if absolute_column_index > 0 and tile[index][absolute_column_index] == tile[index][absolute_column_index-1]:
                                tile[index][absolute_column_index-1] = -1

        print(tile)
        all_tiles.append(tile)

    socket_to_tiles[socket] = all_tiles

In [ ]:
def print_tikz(cha_matrix: typing.List[typing.List[np.int64]], tile_index: int, cha_to_cpu_map: dict) -> typing.NoReturn:
    updated_cha_matrix = copy.deepcopy(cha_matrix)

    for row_index, row in enumerate(updated_cha_matrix):
        for column_index, column in enumerate(row):
            if column == -1:
                # Fill missing spaces with X
                updated_cha_matrix[row_index][column_index] = "X"
            else:
                # Fill CHAs with CPUs
                if column in cha_to_cpu_map:
                    updated_cha_matrix[row_index][column_index] = f"{column}/{cha_to_cpu_map[column]}"
                else:
                    updated_cha_matrix[row_index][column_index] = f"{column}/X"

    x_offset = 0
    y_offset = 0

    # Fill missing memory controllers
    if tile_index == 0:
        updated_cha_matrix[0][1] = "M"
        y_offset += 8
        print(f"""\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (0.0,{y_offset+6.4:.1f}) {{Intel UPI}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (1.6,{y_offset+6.4:.1f}) {{Accel.}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (3.2,{y_offset+6.4:.1f}) {{Flex Bus}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (4.8,{y_offset+6.4:.1f}) {{Flex Bus}};""")
    elif tile_index == 1:
        updated_cha_matrix[0][2] = "M"
        print("""\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (0.0,0.0) {Intel UPI};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (1.6,0.0) {Accel.};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (3.2,0.0) {Flex Bus};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at (4.8,0.0) {Flex Bus};""")
        y_offset += 1.6
    elif tile_index == 2:
        updated_cha_matrix[3][1] = "M"
        y_offset += 8
        x_offset += 6.4
        print(f"""\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+4.8:.1f},{y_offset+6.4:.1f}) {{Intel UPI}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+3.2:.1f},{y_offset+6.4:.1f}) {{Accel.}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+1.6:.1f},{y_offset+6.4:.1f}) {{Flex Bus}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset:.1f},{y_offset+6.4:.1f}) {{Flex Bus}};""")
    elif tile_index == 3:
        updated_cha_matrix[3][2] = "M"
        x_offset += 6.4
        print(f"""\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+4.8:.1f},0.0) {{Intel UPI}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+3.2:.1f},0.0) {{Accel.}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+1.6:.1f},0.0) {{Flex Bus}};
\\node[draw,text width=1cm,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset:.1f},0.0) {{Flex Bus}};""")
        y_offset += 1.6
    else:
        assert("Tile indices bigger that 3 are not valid")

    for row_index, row in enumerate(updated_cha_matrix):
        for column_index, column in enumerate(reversed(row)):
            column_pos = row_index * 1.6
            row_pos = column_index * 1.6
            print(f"\\node[draw,minimum height=1.5cm,minimum width=1.5cm,anchor=center] at ({x_offset+column_pos:.1f},{y_offset+row_pos:.1f}) {{{column}}};")
            # print(row_index, column_index, column)

In [ ]:
def save_capture(capture, filename: str):
    save_dir = Plotting.create_save_dir(experiment.experiment_name)
    with open(save_dir / filename, 'w') as f:
        f.write(capture.stdout)

In [ ]:
%%capture socket0_cap --no-stderr

print("""\\documentclass{standalone}
\\usepackage{tikz}

\\begin{document}
\\begin{tikzpicture}""")

for tile_index, tile in enumerate(all_tiles):
    print_tikz(tile, tile_index, get_cha_to_cpu_map(socket=0))

print("""\\end{tikzpicture}
\\end{document}""")

In [ ]:
%%capture socket1_cap --no-stderr

print("""\\documentclass{standalone}
\\usepackage{tikz}

\\begin{document}
\\begin{tikzpicture}""")

for tile_index, tile in enumerate(all_tiles):
    print_tikz(tile, tile_index, get_cha_to_cpu_map(socket=1))

print("""\\end{tikzpicture}
\\end{document}""")

In [ ]:
%%capture placement_cap --no-stderr

for socket_index, all_tiles in socket_to_tiles.items():
    for tile_index, tile in enumerate(all_tiles):
        print(f"Tile {tile_index} of socket {socket_index}:")
        print_tile(tile, tile_index, get_cha_to_cpu_map(socket_index))

In [ ]:
print(socket0_cap.stdout)

In [ ]:
save_capture(socket0_cap, 'socket0_tikz.tikz')
save_capture(socket1_cap, 'socket1_tikz.tikz')
save_capture(placement_cap, 'core_and_cha_placement.txt')